GSE113113

In [1]:
import pandas as pd
from ExpressionTools import pyXset, EsetPrep
from rpy2.robjects.packages import importr
from rpy2.robjects import r


/Users/coltongarelli/anaconda3/envs/ExprArch/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
R[write to console]: No methods found in package 'RSQLite' for request: 'dbListFields' when loading 'oligo'

R[write to console]: No methods found in package 'RSQLite' for request: 'dbListFields' when loading 'lumi'



In [2]:
prep = EsetPrep.GeoPrep("GSE113113", "data")
prep.load_geo()

R[write to console]: Found 1 file(s)

R[write to console]: GSE113113_series_matrix.txt.gz

R[write to console]: Using locally cached version: data/GSE113113_series_matrix.txt.gz

R[write to console]: Parsed with column specification:
cols(
  .default = col_character()
)

R[write to console]: See spec(...) for full column specifications.

R[write to console]: Annotation GPL not available, so will use submitter GPL instead

R[write to console]: Using locally cached version of GPL16791 found here:
data/GPL16791.soft 



In [3]:
geoquery = importr('GEOquery')
geoquery.getGEOSuppFiles("GSE113113", baseDir="data")

R[write to console]: trying URL 'https://ftp.ncbi.nlm.nih.gov/geo/series/GSE113nnn/GSE113113/suppl//GSE113113_data_final.txt.gz?tool=geoquery'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 5084571 bytes (4.8 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to c

,size,isdir,mode,mtime,ctime,atime,uid,gid,uname,grname
data/GSE113113/GSE113113_data_final.txt.gz,5084571.0,0,420,1.590692e+09,1.590692e+09,1.590692e+09,501,20,coltongarelli,staff


In [4]:
data = pd.read_csv("data/GSE113113/GSE113113_data_final.txt", sep='\t')

In [5]:
data.set_index('REFID', inplace=True)
# base = importr('base')

In [6]:
from ExpressionTools import pyEset
from rpy2.robjects import StrVector
biobase = importr('Biobase')
data_table = importr('data.table')

combined = biobase.ExpressionSet(assayData=r.matrix(data.to_numpy(), ncol=len(data.columns), nrow=len(data.index)),
                                 featureData=biobase.AnnotatedDataFrame(pd.DataFrame(data.index)),
                                 phenoData=biobase.AnnotatedDataFrame(prep.eset.pheno_data),
                                 protocolData=prep.eset.protocol_data,
                                 annotation=StrVector(prep.eset.annot))
xset = pyXset.PyXset(pyEset.PyEset(combined))

/Users/coltongarelli/anaconda3/envs/ExprArch/lib/python3.7/site-packages/pandas/core/strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [ ]:
early = xset.search_pdata(["IIB", "IA"], "Early stage", "characteristics_ch1.2")
late = xset.search_pdata("IV", "Late stage", "characteristics_ch1.2")
healthy = xset.search_pdata("Normal", "Healthy", "characteristics_ch1.2")

In [7]:
xset.update_groups(early)
xset.update_groups(late)
xset.update_groups(healthy)

In [8]:
xset.exprs.index = xset.fdata.loc[:, 'REFID']

In [9]:
from AnalysisTools import gsea

In [10]:
early_analyze = xset.exprs.loc[:, early]
early_analyze = early_analyze.join(xset.exprs.loc[:, healthy])
early_gsea = gsea.GSEAnalysis(".", '/GSE113113/gsea/early')
early_classes = early_gsea.construct_classes('Early stage', 'Healthy', len([i for i in early.to_list() if i]), len([i for i in healthy.to_list() if i]))
# early_gsea.run_gsea(early_analyze, early_classes)
late_analyze = xset.exprs.loc[:, late]
late_analyze = late_analyze.join(xset.exprs.loc[:, healthy])
late_gsea = gsea.GSEAnalysis(".", '/GSE113113/gsea/late')
late_classes = late_gsea.construct_classes('Late stage', 'Healthy', len([i for i in late.to_list() if i]), len([i for i in healthy.to_list() if i]))
# late_gsea.run_gsea(late_analyze, late_classes)

Differential expression


In [11]:
from AnalysisTools import limma_de
from importlib import reload

In [16]:
# %load_ext autoreload
# %autoreload 1
de = limma_de.LimmaDiffEx(xset)
de.run_contrast_fit()
de.run_ebayes()
tops = de.run_toptable('p')

AttributeError: module 'limma' has no attribute 'ebayes'

In [15]:
de